.

# Menggunakan Model :   
*   Linear Regression
*   Decision Tree Regressor
*   Random Forest Regressor
*   XGB Regressor
*   Gradient Boosting Regressor
*   LGBM Regressor
*   SVR


# Load Data

In [5]:
# from google.colab import files

# uploaded = files.upload()

Saving Data Harga Rumah di Jakarta - All Data.csv to Data Harga Rumah di Jakarta - All Data.csv


In [6]:
import pandas as pd
# data = pd.read_csv(next(iter(uploaded)))
# jika memakai jupyter ubah comment yang atas dan uncomment yang bawah
data = pd.read_csv("Data Harga Rumah di Jakarta - All Data.csv")
len_baris = len(data)
print(data.head())

   No      NRP                Nama       harga rumah jumlah kamar tidur  \
0   1  1572040           Rossevine   Rp2,450,000,000                  4   
1   2  2272018  Nathanael Kanaya C  Rp19,000,000,000                  7   
2   3  2272018  Nathanael Kanaya C   Rp3,500,000,000                  4   
3   4  2272018  Nathanael Kanaya C  Rp67,000,000,000                  5   
4   5  2272018  Nathanael Kanaya C  Rp13,300,000,000                  4   

  jumlah kamar mandi luas tanah (m2)  luas bangunan (m2) carport (mobil)  \
0                  2             105                 104               1   
1                  4             633                 400               3   
2                  4             121                 153               1   
3                  6            2550                1107               8   
4                  2             706                 708               5   

  pasokan listrik (watt)   Kabupaten/Kota   kecamatan   kelurahan  \
0                   220

# Libraries

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

# Preprocess Data

In [8]:
# hapus kolom data yang tidak dipakai
columns_to_drop = ['No', 'NRP', 'Nama', 'kelurahan', 'Link Data Rumah']
data.drop(columns=[col for col in columns_to_drop if col in data.columns], inplace=True)

In [9]:
# hapus data yang paling kanan Unnamed: 19 * contoh data
data.drop(columns=['Unnamed: 19'], inplace=True)

data.fillna(method='ffill', inplace=True)

In [10]:
# melakukan replace pada ',' karena komputer hanya membaca '.' sebagai decimal
# dikarenakan Amerika Serikat yang menemukan komputer terlebih dahulu yang memakai pemisah desimal '.'
for col in ['jarak dengan rumah sakit terdekat (km)',
            'jarak dengan sekolah terdekat (km)', 'jarak dengan tol terdekat (km)']:
    data[col] = data[col].str.replace(',', '.', regex=True).astype(float)


In [11]:
# dengan berpikir dengan pattern recognition saya melihat ada yang memakai ',' dan '.' dan juga ada yang memakai .00 dan "Rp " dan "Rp"
# dengan demikian saya menghapus 3 karakter jika memiliki dibelakanya ".00" kemudian replace ',' dan '.' menjadi '' sehingga string dapat
# menjadi satu menjadi intger, dan juga membuang 'Rp' dan 'Rp ' menjadi ''
def clean_and_convert(index, amount_str):
    # menghapus RP '.' ',' dengan membuang belakang .00 dan menghitung jumlah 0 nya

    text = (amount_str)

    if text.endswith(".00"):
        text = text[:-3]
    else:
        pass
    text = text.replace('.', '').replace(',', '')
    text = text.replace('Rp', '').replace('Rp ', '')
    text = text.replace(' ', '')
    return int(text)

data['harga rumah'] = data.apply(lambda row: clean_and_convert(row.name, row['harga rumah']), axis=1)

In [12]:
# Melakukkan Pengecekan data yang ingin di one hot
def cek_data(colom):
    uniq = []
    for value in data[colom]:
        if value not in uniq:
            uniq.append(value)
    print("Unique values", colom)
    print(uniq)

cek_data("keamanan (ada/tidak)")

cek_data("taman (ada/tidak)")

cek_data("Kabupaten/Kota")

Unique values keamanan (ada/tidak)
['tidak', 'ada', 'tidak ', 'ya', 'tdk', 'ada ', 'Ada', 'Tidak', 'Tidak ada', 'Tidak ']
Unique values taman (ada/tidak)
['tidak', 'ada', 'tdk', 'ya', 'tiadk', 'Tidak', 'Ada', 'Tidak ada', 'TIdak', 'tdak', ' tidak', 'tidak ', 'tdiak']
Unique values Kabupaten/Kota
['Jakarta Selatan', 'Jakarta Timur', 'Jakarta Pusat', 'Jakarta Utara', 'Jakarta Barat']


In [13]:
# ubah menjadi semua data menjadi huruf kecil
data['keamanan (ada/tidak)'] = data['keamanan (ada/tidak)'].str.lower()
data['taman (ada/tidak)'] = data['taman (ada/tidak)'].str.lower()

In [14]:
# Dengan memakai pattern recognition saya melihat bahwa semua data tidak serupa jadi saya meilhat persamaannya
# ternyata yang string artinya 'tidak' seperti 'tdk', 'tdak', dll. memiliki awalan huruf 't' dan 'ada' tidak memiliki awalan 't'

# sebelum itu hapus terlebih dahulu data yang kosong
data = data[data['keamanan (ada/tidak)'].notna() & (data['keamanan (ada/tidak)'] != '')]
data = data[data['taman (ada/tidak)'].notna() & (data['taman (ada/tidak)'] != '')]

def map_tidak(kata):
    if kata[0].lower() == 't':
        return 'tidak'
    else:
        return 'ada'

# mengubah semuannya menjadi tidak dan ada
data['keamanan (ada/tidak)'] = data['keamanan (ada/tidak)'].apply(map_tidak)
data['taman (ada/tidak)'] = data['taman (ada/tidak)'].apply(map_tidak)

In [15]:
# mengecek data lainnya
cek_data("jumlah kamar tidur")
cek_data("jumlah kamar mandi")
cek_data("luas tanah (m2)")
cek_data("luas bangunan (m2)")
cek_data("carport (mobil)")
cek_data("pasokan listrik (watt)")
cek_data("jarak dengan rumah sakit terdekat (km)")
cek_data("jarak dengan sekolah terdekat (km)")
cek_data("jarak dengan tol terdekat (km)")
cek_data("kecamatan")

Unique values jumlah kamar tidur
['4', '7', '5', '3', '6', '2', '10', '8', '13', '9', '24', '12', '26', '16', '17', '15', '1', '19', '11', '20', '73', '14', '25', '0', '30', '18', '23', '-']
Unique values jumlah kamar mandi
['2', '4', '6', '3', '5', '7', '9', '8', '1', '24', '11', '13', '10', '12', '26', '18', '19', '50', '14', '0', '62', '30', '22', '-']
Unique values luas tanah (m2)
['105', '633', '121', '2550', '706', '150', '180', '400', '200', '300', '350', '800', '500', '160', '250', '220', '270', '340', '210', '450', '320', '380', '540', '860', '900', '590', '10000', '760', '890', '560', '65', '40', '196', '75', '135', '895', '356', '366', '119', '990', '90', '264', '512', '471', '831', '654', '1000', '260', '118', '194', '130', '170', '436', '1040', '50', '138', '101', '371', '266', '464', '1402', '92', '124', '98', '76', '577', '1460', '893', '431', '114', '323', '184', '123', '71', '140', '120', '84', '168', '148', '126', '235', '102', '96', '153', '212', '141', '147', '308',

In [16]:
# menghapus data yang tidak ada isinya '' / null
data = data[data['jumlah kamar tidur'].notna() & (data['jumlah kamar tidur'] != '')]
data = data[data['jumlah kamar mandi'].notna() & (data['jumlah kamar mandi'] != '')]
data = data[data['luas tanah (m2)'].notna() & (data['luas tanah (m2)'] != '')]
data = data[data['luas bangunan (m2)'].notna() & (data['luas bangunan (m2)'] != '')]
data = data[data['pasokan listrik (watt)'].notna() & (data['pasokan listrik (watt)'] != '')]
data = data[data['jarak dengan rumah sakit terdekat (km)'].notna() & (data['jarak dengan rumah sakit terdekat (km)'] != '')]
data = data[data['jarak dengan sekolah terdekat (km)'].notna() & (data['jarak dengan sekolah terdekat (km)'] != '')]
data = data[data['jarak dengan tol terdekat (km)'].notna() & (data['jarak dengan tol terdekat (km)'] != '')]
data = data[data['carport (mobil)'].notna() & (data['carport (mobil)'] != '')]
data = data[data['kecamatan'].notna() & (data['kecamatan'] != '')]

In [17]:
# mustahil untuk rumah tidak memiliki luas tanah (m2), luas bangunan (m2)
data = data[(data['luas tanah (m2)'] != '0') & (data['luas tanah (m2)'] != 0)]
data = data[(data['luas bangunan (m2)'] != '0') & (data['luas tanah (m2)'] != 0)]

# Sangat Jarang juga rumah yang tidak memiliki pasokan listrik
data = data[(data['pasokan listrik (watt)'] != '0') & (data['pasokan listrik (watt)'] != 0) & (data['pasokan listrik (watt)'] != '-') & (data['pasokan listrik (watt)'] != '--')]

# terdapat ',' didata jadi dihilangkan untuk dapat di convert ke integer
data['pasokan listrik (watt)'] = data['pasokan listrik (watt)'].astype(str).str.replace(',', '')

# '-' mungkin isisnya 0 dan
data['jumlah kamar tidur'] = data['jumlah kamar tidur'].replace('-', '0')
data['jumlah kamar mandi'] = data['jumlah kamar mandi'].replace('-', '0')
data['carport (mobil)'] = data['carport (mobil)'].replace('-', '0')

In [18]:
# Ubah terlebih dahulu tipe data jika data berupa numerik
data['jumlah kamar tidur'] = data['jumlah kamar tidur'].astype(int)
data['jumlah kamar mandi'] = data['jumlah kamar mandi'].astype(int)
data['luas tanah (m2)'] = data['luas tanah (m2)'].astype(int)
data['luas bangunan (m2)'] = data['luas bangunan (m2)'].astype(int)
data['carport (mobil)'] = data['carport (mobil)'].astype(int)
data['pasokan listrik (watt)'] = data['pasokan listrik (watt)'].astype(int)

In [19]:
# huruf pertama pada kecamatan diganti menjadi besar
data['kecamatan'] = data['kecamatan'].apply(lambda name: ' '.join([part.capitalize() for part in name.split()]))

In [20]:
# Entah Mengapa ada 2 penulisan Pasar Rebo: yaitu Ps. Rebo dan Pasar Rebo
# Jadi saya ubah Ps. Rebo menjadi Pasar Rebo
data['kecamatan'] = data['kecamatan'].replace('Ps. Rebo', 'Pasar Rebo')

In [21]:
# cek ulang data
cek_data("keamanan (ada/tidak)")
cek_data("taman (ada/tidak)")
cek_data("Kabupaten/Kota")
cek_data("jumlah kamar tidur")
cek_data("jumlah kamar mandi")
cek_data("luas tanah (m2)")
cek_data("luas bangunan (m2)")
cek_data("carport (mobil)")
cek_data("pasokan listrik (watt)")
cek_data("jarak dengan rumah sakit terdekat (km)")
cek_data("jarak dengan sekolah terdekat (km)")
cek_data("jarak dengan tol terdekat (km)")
cek_data("kecamatan")

Unique values keamanan (ada/tidak)
['tidak', 'ada']
Unique values taman (ada/tidak)
['tidak', 'ada']
Unique values Kabupaten/Kota
['Jakarta Selatan', 'Jakarta Timur', 'Jakarta Pusat', 'Jakarta Utara', 'Jakarta Barat']
Unique values jumlah kamar tidur
[4, 7, 5, 3, 6, 2, 10, 8, 13, 9, 24, 12, 26, 16, 17, 15, 1, 19, 11, 20, 73, 14, 25, 0, 30, 18, 23]
Unique values jumlah kamar mandi
[2, 4, 6, 3, 5, 7, 9, 8, 1, 24, 11, 13, 10, 12, 26, 18, 19, 50, 14, 0, 62, 30, 22]
Unique values luas tanah (m2)
[105, 633, 121, 2550, 706, 150, 180, 400, 200, 300, 350, 800, 500, 160, 250, 220, 270, 340, 210, 450, 320, 380, 540, 860, 900, 590, 10000, 760, 890, 560, 65, 40, 196, 75, 135, 895, 356, 366, 119, 990, 90, 264, 512, 471, 831, 654, 1000, 260, 118, 194, 130, 170, 436, 1040, 50, 138, 101, 371, 266, 464, 1402, 92, 124, 98, 76, 577, 1460, 893, 431, 114, 323, 184, 123, 71, 140, 120, 84, 168, 148, 126, 235, 102, 96, 153, 212, 141, 147, 308, 528, 198, 30, 61, 89, 70, 181, 352, 480, 193, 563, 91, 122, 68, 112

In [22]:
# kalkulasi Jumlah data awal dan akhir
print('perhitungan jumlah data perbaris yang sudah di Data preprocessing atau data preparation')
ak = len(data)
print('data akhir:', ak)
print('data awal:', len_baris)
print('Total penghapusan', len_baris - ak)

perhitungan jumlah data perbaris yang sudah di Data preprocessing atau data preparation
data akhir: 2331
data awal: 2410
Total penghapusan 79


In [23]:
# penghapusan data yang mustahi;
data = data[data["jarak dengan rumah sakit terdekat (km)"] <= 50]
data = data[data["jarak dengan sekolah terdekat (km)"] <= 50]
data = data[data["jarak dengan tol terdekat (km)"] <= 50]

In [24]:
# sampai sini bernard bertrand

# kemudian Jason mulai dari sini sampai best model skor yang pertama

In [25]:
print("data yang dihapus: ")
print('data akhir:', len(data))
print('data awal:', ak)
print('Total penghapusan', ak - len(data))

data yang dihapus: 
data akhir: 2295
data awal: 2331
Total penghapusan 36


In [ ]:
y = data['harga rumah']
X = data.drop(columns=['harga rumah'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

,jumlah kamar tidur,jumlah kamar mandi,luas tanah (m2),luas bangunan (m2),carport (mobil),pasokan listrik (watt),Kabupaten/Kota,kecamatan,keamanan (ada/tidak),taman (ada/tidak),jarak dengan rumah sakit terdekat (km),jarak dengan sekolah terdekat (km),jarak dengan tol terdekat (km)
0,4,2,105,104,1,2200,Jakarta Selatan,Gedebage,tidak,tidak,2.5,0.3,2.0
1,7,4,633,400,3,10000,Jakarta Selatan,Pengadegan,ada,ada,0.9,1.9,1.9
2,4,4,121,153,1,2200,Jakarta Selatan,Pengadegan,tidak,ada,1.0,1.8,1.8
3,5,6,2550,1107,8,7700,Jakarta Selatan,Cikoko,ada,ada,0.8,2.0,2.0
4,4,2,706,708,5,4400,Jakarta Selatan,Rawajati,tidak,ada,1.2,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,4,3,198,350,2,5500,Jakarta Utara,Tanjung Priok,ada,ada,1.1,1.0,5.0
2405,4,2,102,160,0,2200,Jakarta Utara,Kelapa Gading,tidak,ada,1.6,1.8,4.0
2406,3,3,230,400,2,4400,Jakarta Utara,Penjaringan,ada,ada,1.7,5.8,3.5
2408,3,3,230,450,1,7700,Jakarta Utara,Penjaringan,ada,ada,2.0,5.5,3.0


In [ ]:
y

0        2450000000
1       19000000000
2        3500000000
3       67000000000
4       13300000000
           ...     
2403     6200000000
2405     2800000000
2406    10900000000
2408    11000000000
2409    18500000000
Name: harga rumah, Length: 2295, dtype: int64

In [ ]:
print(data.columns)

Index(['harga rumah', 'jumlah kamar tidur', 'jumlah kamar mandi',
       'luas tanah (m2)', 'luas bangunan (m2)', 'carport (mobil)',
       'pasokan listrik (watt)', 'Kabupaten/Kota', 'kecamatan',
       'keamanan (ada/tidak)', 'taman (ada/tidak)',
       'jarak dengan rumah sakit terdekat (km)',
       'jarak dengan sekolah terdekat (km)', 'jarak dengan tol terdekat (km)'],
      dtype='object')


# Models dan Hyperparameter Tuning

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
categorical_features = ['Kabupaten/Kota', 'kecamatan', 'keamanan (ada/tidak)', 'taman (ada/tidak)']
numerical_features = data.columns.difference(categorical_features + ['harga rumah']).tolist()


poly = PolynomialFeatures(degree=2, include_bias=False)
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('poly', poly)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [ ]:

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'LightGBM': lgb.LGBMRegressor(),
    'SVR': SVR()
}

# Model nya underfitting jadi parameternya saya ganti
param_grid = {
    'RandomForest': {
        'model__n_estimators': [50, 100, 200, 500],
        'model__max_depth': [10, 20, 30, None],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    },
    'XGBRegressor': {
        'model__n_estimators': [50, 100, 200, 500],
        'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
        'model__max_depth': [3, 6, 9]
    },
    'GradientBoosting': {
        'model__n_estimators': [50, 100, 200, 500],
        'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
        'model__max_depth': [3, 4, 5]
    },
    'LightGBM': {
        'model__n_estimators': [50, 100, 200, 500],
        'model__learning_rate': [0.01, 0.1, 0.2, 0.3],
        'model__num_leaves': [31, 50, 100]
    },
    'SVR': {
        'model__C': [0.1, 1, 10, 100],
        'model__epsilon': [0.01, 0.1, 0.2],
        'model__kernel': ['linear', 'rbf']
    }
}

best_models = {}
for model_name in models:
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', models[model_name])])

    if model_name in param_grid:
        search = GridSearchCV(model, param_grid[model_name], cv=5, scoring='r2', n_jobs=-1)
    else:
        search = model

    search.fit(X_train, y_train)
    best_models[model_name] = search.best_estimator_ if model_name in param_grid else model


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000087 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 986
[LightGBM] [Info] Number of data points in the train set: 1836, number of used features: 42
[LightGBM] [Info] Start training from score 11333797927.564270


# Evaluasi Data

In [ ]:

results = {}
for model_name in best_models:
    model = best_models[model_name]
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    results[model_name] = {'R²': r2, 'MAE': mae, 'RMSE': rmse}

results_df = pd.DataFrame(results).T
print(results_df)

                        R²           MAE          RMSE
LinearRegression  0.066900  1.078863e+10  1.015939e+11
DecisionTree      0.120368  1.014766e+10  9.864016e+10
RandomForest      0.103692  9.159154e+09  9.957082e+10
XGBRegressor      0.061485  1.020839e+10  1.018882e+11
GradientBoosting  0.096768  9.159040e+09  9.995465e+10
LightGBM          0.085906  9.472517e+09  1.005539e+11
SVR              -0.015994  1.553304e+10  1.060105e+11


# Best Model dan Hasil

In [ ]:
best_model_name = results_df['R²'].idxmax()
best_model = best_models[best_model_name]

print(f"The best model is {best_model_name} with an R² score of {results_df.loc[best_model_name, 'R²']:.2f}")

y_pred = best_model.predict(X_test)

comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison_df.head())

The best model is DecisionTree with an R² score of 0.12
           Actual     Predicted
424   13500000000  7.000000e+09
1326   5100000000  4.790000e+09
209   26000000000  1.600000e+10
1670   1800000000  1.600000e+09
1665  15000000000  1.000000e+10


In [ ]:
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(best_model, X, y, cv=cv, scoring='r2')

print(f"Cross-validated R² score: {cv_results.mean():.2f} ± {cv_results.std():.2f}")

Cross-validated R² score: -4.18 ± 13.52


# Model Terbaik = Random Forest

In [ ]:
best_model.score(X_test, y_test)

0.1203683523006861

In [ ]:
# sini hubert sampai akhir

In [ ]:
# import joblib
# model_filename = f"{best_model_name}_model.joblib"
# joblib.dump(best_model, model_filename)
# print(f"Model saved as {model_filename}")


Coba Lagi tetapi dengan menghapus outlier

In [ ]:

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define the outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DataFrame to remove outliers
    df_no_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    # Calculate number of outliers removed
    outliers_removed = len(df) - len(df_no_outliers)

    print(outliers_removed)
    return df_no_outliers

data = remove_outliers(data, 'harga rumah')
data = remove_outliers(data, 'jarak dengan rumah sakit terdekat (km)')
data = remove_outliers(data, 'jarak dengan sekolah terdekat (km)')

279
86
74


In [ ]:
print("outliers yang dihapus: ")
print('Total data akhir:', len(data))

outliers yang dihapus: 
Total data akhir: 1856


In [ ]:
y = data['harga rumah']
X = data.drop(columns=['harga rumah'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
best_models = {}
for model_name in models:
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', models[model_name])])

    if model_name in param_grid:
        search = GridSearchCV(model, param_grid[model_name], cv=5, scoring='r2', n_jobs=-1)
    else:
        search = model

    search.fit(X_train, y_train)
    best_models[model_name] = search.best_estimator_ if model_name in param_grid else model

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 874
[LightGBM] [Info] Number of data points in the train set: 1484, number of used features: 41
[LightGBM] [Info] Start training from score 5707713609.358491


In [ ]:
results = {}
for model_name in best_models:
    model = best_models[model_name]
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    results[model_name] = {'R²': r2, 'MAE': mae, 'RMSE': rmse}

results_df = pd.DataFrame(results).T
print(results_df)

                        R²           MAE          RMSE
LinearRegression  0.670845  1.609173e+09  2.552761e+09
DecisionTree      0.498239  1.895188e+09  3.151799e+09
RandomForest      0.752638  1.409161e+09  2.212977e+09
XGBRegressor      0.752718  1.388277e+09  2.212617e+09
GradientBoosting  0.765138  1.352543e+09  2.156338e+09
LightGBM          0.722599  1.504310e+09  2.343497e+09
SVR              -0.088633  3.103659e+09  4.642488e+09


In [ ]:

best_model_name = results_df['R²'].idxmax()
best_model = best_models[best_model_name]

print(f"The best model is {best_model_name} with an R² score of {results_df.loc[best_model_name, 'R²']:.2f}")

y_pred = best_model.predict(X_test)

comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(comparison_df.head())

The best model is GradientBoosting with an R² score of 0.77
          Actual     Predicted
253   2900000000  2.522996e+09
654   5500000000  4.041784e+09
1514  5500000000  3.336336e+09
1666  2300000000  3.165913e+09
608   6500000000  4.287136e+09


In [ ]:
cv = KFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(best_model, X, y, cv=cv, scoring='r2')

print(f"Cross-validated R² score: {cv_results.mean():.2f} ± {cv_results.std():.2f}")

Cross-validated R² score: 0.76 ± 0.05


In [ ]:
print("model terbaik setelah hapus data yang mustahil: ",best_model.score(X_test, y_test))

model terbaik setelah hapus data yang mustahil:  0.7651375295790583
